In [ ]:
# import rasterio
# import rasterio.warp
# import rasterio.fill # Might be needed for nodata handling, but often simple replace is enough
import numpy as np
# import geopandas as gpd
# from shapely.geometry import Point
# import pandas as pd
from osgeo import gdal

In [9]:
input_dem_path = '../../Documents/soilense_data/DEM/q16.dem'

In [10]:
dem_dataset = gdal.Open(input_dem_path)

In [5]:
if dem_dataset:
    # Define output slope file
    output_slope_path = "q909_slope_output.tif"

    # Process DEM to calculate slope
    # Options can be passed as a list of strings, e.g., options=['-p'] for percent slope
    slope_dataset = gdal.DEMProcessing(output_slope_path,
                                       dem_dataset,
                                       "slope",
                                       computeEdges=True)
                                       # Example for percent slope: options=gdal.DEMProcessingOptions(options=['-p']))


    if slope_dataset:
        slope_array = slope_dataset.GetRasterBand(1).ReadAsArray()
        print("Slope calculation successful. Mean slope:", slope_array.mean())

        # Close datasets
        slope_dataset = None
        dem_dataset = None
    else:
        print("Error: Could not process DEM for slope.")
else:
    print(f"Error: Could not open DEM file {input_dem_path}")

Slope calculation successful. Mean slope: -239.50659


In [11]:
geotransform = dem_dataset.GetGeoTransform()

In [13]:
from dem_processor.utils import get_dem_info

SyntaxError: invalid syntax (utils.py, line 98)

In [ ]:
geotransform[0] = top-left X
geotransform[1] = W-E pixel resolution (pixel width)
geotransform[2] = row rotation (typically 0)
geotransform[3] = top-left Y
geotransform[4] = column rotation (typically 0)